In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import io, s3fs, json, tabula
pd.set_option('display.max_columns', None)
print('Program run at', dt.now())

Program run at 2021-05-20 18:07:21.364725


This program reads in ACS and COI data from the S3 bucket and renames columns with descriptive titles.

# Rename ACS columns  
Import ACS data and header data from AWS.

In [2]:
acs = pd.read_csv('s3://bleeding-hearts/rawdata/acs5.csv',
                 low_memory=False,
                 error_bad_lines=False)
acs.head()

b'Skipping line 35937: expected 1103 fields, saw 1795\n'


,Unnamed: 0,GEO_ID,DP03_0001E,DP03_0001M,DP03_0001PE,DP03_0001PM,DP03_0002E,DP03_0002M,DP03_0002PE,DP03_0002PM,DP03_0003E,DP03_0003M,DP03_0003PE,DP03_0003PM,DP03_0004E,DP03_0004M,DP03_0004PE,DP03_0004PM,DP03_0005E,DP03_0005M,DP03_0005PE,DP03_0005PM,DP03_0006E,DP03_0006M,DP03_0006PE,DP03_0006PM,DP03_0007E,DP03_0007M,DP03_0007PE,DP03_0007PM,DP03_0008E,DP03_0008M,DP03_0008PE,DP03_0008PM,DP03_0009E,DP03_0009M,DP03_0009PE,DP03_0009PM,DP03_0010E,DP03_0010M,DP03_0010PE,DP03_0010PM,DP03_0011E,DP03_0011M,DP03_0011PE,DP03_0011PM,DP03_0012E,DP03_0012M,DP03_0012PE,DP03_0012PM,DP03_0013E,DP03_0013M,DP03_0013PE,DP03_0013PM,DP03_0014E,DP03_0014M,DP03_0014PE,DP03_0014PM,DP03_0015E,DP03_0015M,DP03_0015PE,DP03_0015PM,DP03_0016E,DP03_0016M,DP03_0016PE,DP03_0016PM,DP03_0017E,DP03_0017M,DP03_0017PE,DP03_0017PM,DP03_0018E,DP03_0018M,DP03_0018PE,DP03_0018PM,DP03_0019E,DP03_0019M,DP03_0019PE,DP03_0019PM,DP03_0020E,DP03_0020M,DP03_0020PE,DP03_0020PM,DP03_0021E,DP03_0021M,DP03_0021PE,DP03_0021PM,DP03_0022E,DP03_0022M,DP03_0022PE,DP03_0022PM,DP03_0023E,DP03_0023M,DP03_0023PE,DP03_0023PM,DP03_0024E,DP03_0024M,DP03_0024PE,DP03_0024PM,DP03_0025E,DP03_0025M,DP03_0025PE,DP03_0025PM,DP03_0026E,DP03_0026M,DP03_0026PE,DP03_0026PM,DP03_0027E,DP03_0027M,DP03_0027PE,DP03_0027PM,DP03_0028E,DP03_0028M,DP03_0028PE,DP03_0028PM,DP03_0029E,DP03_0029M,DP03_0029PE,DP03_0029PM,DP03_0030E,DP03_0030M,DP03_0030PE,DP03_0030PM,DP03_0031E,DP03_0031M,DP03_0031PE,DP03_0031PM,DP03_0032E,DP03_0032M,DP03_0032PE,DP03_0032PM,DP03_0033E,DP03_0033M,DP03_0033PE,DP03_0033PM,DP03_0034E,DP03_0034M,DP03_0034PE,DP03_0034PM,DP03_0035E,DP03_0035M,DP03_0035PE,DP03_0035PM,DP03_0036E,DP03_0036M,DP03_0036PE,DP03_0036PM,DP03_0037E,DP03_0037M,DP03_0037PE,DP03_0037PM,DP03_0038E,DP03_0038M,DP03_0038PE,DP03_0038PM,DP03_0039E,DP03_0039M,DP03_0039PE,DP03_0039PM,DP03_0040E,DP03_0040M,DP03_0040PE,DP03_0040PM,DP03_0041E,DP03_0041M,DP03_0041PE,DP03_0041PM,DP03_0042E,DP03_0042M,DP03_0042PE,DP03_0042PM,DP03_0043E,DP03_0043M,DP03_0043PE,DP03_0043PM,DP03_0044E,DP03_0044M,DP03_0044PE,DP03_0044PM,DP03_0045E,DP03_0045M,DP03_0045PE,DP03_0045PM,DP03_0046E,DP03_0046M,DP03_0046PE,DP03_0046PM,DP03_0047E,DP03_0047M,DP03_0047PE,DP03_0047PM,DP03_0048E,DP03_0048M,DP03_0048PE,DP03_0048PM,DP03_0049E,DP03_0049M,DP03_0049PE,DP03_0049PM,DP03_0050E,DP03_0050PE,DP03_0050M,DP03_0050PM,DP03_0051E,DP03_0051M,DP03_0051PE,DP03_0051PM,DP03_0052E,DP03_0052M,DP03_0052PE,DP03_0052PM,DP03_0053E,DP03_0053M,DP03_0053PE,DP03_0053PM,DP03_0054E,DP03_0054M,DP03_0054PE,DP03_0054PM,DP03_0055E,DP03_0055M,DP03_0055PE,DP03_0055PM,DP03_0056E,DP03_0056M,DP03_0056PE,DP03_0056PM,DP03_0057E,DP03_0057M,DP03_0057PE,DP03_0057PM,DP03_0058E,DP03_0058M,DP03_0058PE,DP03_0058PM,DP03_0059E,DP03_0059M,DP03_0059PE,DP03_0059PM,DP03_0060E,DP03_0060M,DP03_0060PE,DP03_0060PM,DP03_0061E,DP03_0061M,DP03_0061PE,DP03_0061PM,DP03_0062E,DP03_0062M,DP03_0062PE,DP03_0062PM,DP03_0063E,DP03_0063M,DP03_0063PE,DP03_0063PM,DP03_0064E,DP03_0064M,DP03_0064PE,DP03_0064PM,DP03_0065E,DP03_0065M,DP03_0065PE,DP03_0065PM,DP03_0066E,DP03_0066M,DP03_0066PE,DP03_0066PM,DP03_0067E,DP03_0067M,DP03_0067PE,DP03_0067PM,DP03_0068E,DP03_0068M,DP03_0068PE,DP03_0068PM,DP03_0069E,DP03_0069M,DP03_0069PE,DP03_0069PM,DP03_0070E,DP03_0070M,DP03_0070PE,DP03_0070PM,DP03_0071E,DP03_0071M,DP03_0071PE,DP03_0071PM,DP03_0072E,DP03_0072M,DP03_0072PE,DP03_0072PM,DP03_0073E,DP03_0073M,DP03_0073PE,DP03_0073PM,DP03_0074E,DP03_0074M,DP03_0074PE,DP03_0074PM,DP03_0075E,DP03_0075M,DP03_0075PE,DP03_0075PM,DP03_0076E,DP03_0076M,DP03_0076PE,DP03_0076PM,DP03_0077E,DP03_0077M,DP03_0077PE,DP03_0077PM,DP03_0078E,DP03_0078M,DP03_0078PE,DP03_0078PM,DP03_0079E,DP03_0079M,DP03_0079PE,DP03_0079PM,DP03_0080E,DP03_0080M,DP03_0080PE,DP03_0080PM,DP03_0081E,DP03_0081M,DP03_0081PE,DP03_0081PM,DP03_0082E,DP03_0082M,DP03_0082PE,DP03_0082PM,DP03_0083E,DP03_0083M,DP03_0083PE,DP03_0083PM,DP03_0084E,DP03_0084M,DP03_0084PE,DP03_0084PM,DP03_0085E,DP03_0085M,DP03_0085PE,DP03_0085PM,DP03_0086E,DP03_0086M,DP03_0086PE,DP03_0086PM,DP03_0087E,DP03_0087M,DP0

In [3]:
len(acs)

207038

In [4]:
headers = pd.read_csv('s3://bleeding-hearts/rawdata/acs_headers_2015.csv')
headers

Unnamed: 0                                 for  \
0          label        Census API FIPS 'for' clause   
1        concept  Census API Geography Specification   
2  predicateType                            fips-for   
3          group                                 NaN   
4          limit                                   0   
5  predicateOnly                                True   
6     attributes                                 NaN   
7       required                                 NaN   
8         values                                 NaN   

                                   in  \
0         Census API FIPS 'in' clause   
1  Census API Geography Specification   
2                             fips-in   
3                                 NaN   
4                                   0   
5                                True   
6                                 NaN   
7                                 NaN   
8                                 NaN   

                                        ucgid  \
0  Uniform Census Geography Identifier clause   
1          Census API Geography Specification   
2                                       ucgid   
3                                         NaN   
4                                           0   
5                                        True   
6                                         NaN   
7                                         NaN   
8                                         NaN   

                                          DP02_0126E  \
0       Estimate!!ANCESTRY!!Total population!!Danish   
1  SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...   
2                                                int   
3                                               DP02   
4                                                  0   
5                                                NaN   
6                 DP02_0126EA,DP02_0126M,DP02_0126MA   
7                                                NaN   
8                                                NaN   

                                         DP05_0050PE  \
0  Percent!!RACE!!One race!!Native Hawaiian and O...   
1              ACS DEMOGRAPHIC AND HOUSING ESTIMATES   
2                                              float   
3                                               DP05   
4                                                  0   
5                                                NaN   
6              DP05_0050PEA,DP05_0050PM,DP05_0050PMA   
7                                                NaN   
8                                                NaN   

                                          DP04_0047E  \
0  Estimate!!HOUSING TENURE!!Occupied housing uni...   
1                   SELECTED HOUSING CHARACTERISTICS   
2                                                int   
3                                               DP04   
4                                                  0   
5                                                NaN   
6                 DP04_0047EA,DP04_0047M,DP04_0047MA   
7                                                NaN   
8                                                NaN   

                                       DP02PR_0083PE  \
0  Percent!!RESIDENCE 1 YEAR AGO!!Population 1 ye...   
1     SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO   
2                                              float   
3                                             DP02PR   
4                                                  0   
5                                                NaN   
6        DP02PR_0083PEA,DP02PR_0083PM,DP02PR_0083PMA   
7                                                NaN   
8                                                NaN   

                                       DP02PR_0106PE  \
0  Percent!!WORLD REGION OF BIRTH OF FOREIGN BORN...   
1     SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO   
2                                              float   
3                                             DP02PR   
4                  

Turn headers data into dictionary that we can use to rename the columns. Start by transposing the header data and drop all columns except the DP descriptor and the label.

In [5]:
headers=headers.T
headers

,0,1,2,3,4,5,6,7,8
Unnamed: 0,label,concept,predicateType,group,limit,predicateOnly,attributes,required,values
for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,NaN,0,True,NaN,NaN,NaN
in,Census API FIPS 'in' clause,Census API Geography Specification,fips-in,NaN,0,True,NaN,NaN,NaN
ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,ucgid,NaN,0,True,NaN,NaN,NaN
DP02_0126E,Estimate!!ANCESTRY!!Total population!!Danish,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0,NaN,"DP02_0126EA,DP02_0126M,DP02_0126MA",NaN,NaN
...,...,...,...,...,...,...,...,...,...
DP02PR_0001PE,Percent!!HOUSEHOLDS BY TYPE!!Total households,SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO,int,DP02PR,0,NaN,"DP02PR_0001PEA,DP02PR_0001PM,DP02PR_0001PMA",NaN,NaN
DP03_0039PE,Percent!!INDUSTRY!!Civilian employed populatio...,SELECTED ECONOMIC CHARACTERISTICS,float,DP03,0,NaN,"DP03_0039PEA,DP03_0039PM,DP03_0039PMA",NaN,NaN
DP02_0098E,Estimate!!YEAR OF ENTRY!!Native!!Entered 2010 ...,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0,NaN,"DP02_0098EA,DP02_0098M,DP02_0098MA",NaN,NaN
DP04_0095PE,Percent!!SELECTED MONTHLY OWNER COSTS (SMOC)!!...,SELECTED HOUSING CHARACTERISTICS,float,DP04,0,NaN,"DP04_0095PEA,DP04_0095PM,DP04_0095PMA",NaN,NaN


In [6]:
headers1=headers.drop(columns=[1,2,3,4,5,6,7,8])
headers1

,0
Unnamed: 0,label
for,Census API FIPS 'for' clause
in,Census API FIPS 'in' clause
ucgid,Uniform Census Geography Identifier clause
DP02_0126E,Estimate!!ANCESTRY!!Total population!!Danish
...,...
DP02PR_0001PE,Percent!!HOUSEHOLDS BY TYPE!!Total households
DP03_0039PE,Percent!!INDUSTRY!!Civilian employed populatio...
DP02_0098E,Estimate!!YEAR OF ENTRY!!Native!!Entered 2010 ...
DP04_0095PE,Percent!!SELECTED MONTHLY OWNER COSTS (SMOC)!!...


Turn the headers dataframe into a dictionary

In [7]:
headers_d=headers1.to_dict()
headers_d=headers_d[0]
headers_d

{'Unnamed: 0': 'label',
 'for': "Census API FIPS 'for' clause",
 'in': "Census API FIPS 'in' clause",
 'ucgid': 'Uniform Census Geography Identifier clause',
 'DP02_0126E': 'Estimate!!ANCESTRY!!Total population!!Danish',
 'DP05_0050PE': 'Percent!!RACE!!One race!!Native Hawaiian and Other Pacific Islander!!Samoan',
 'DP04_0047E': 'Estimate!!HOUSING TENURE!!Occupied housing units!!Renter-occupied',
 'DP02PR_0083PE': 'Percent!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house in Puerto Rico or the U.S.!!In Puerto Rico!!Different municipio',
 'DP02PR_0106PE': 'Percent!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Africa',
 'DP04_0046E': 'Estimate!!HOUSING TENURE!!Occupied housing units!!Owner-occupied',
 'DP04_0016PE': 'Percent!!YEAR STRUCTURE BUILT!!Total housing units',
 'DP02_0125E': 'Estimate!!ANCESTRY!!Total population!!Czech',
 'DP02_0054PE': 'Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in schoo

Replace the !! with _.

In [8]:
for k,v in headers_d.items():
    headers_d[k]=v.replace('!!','_')
headers_d

{'Unnamed: 0': 'label',
 'for': "Census API FIPS 'for' clause",
 'in': "Census API FIPS 'in' clause",
 'ucgid': 'Uniform Census Geography Identifier clause',
 'DP02_0126E': 'Estimate_ANCESTRY_Total population_Danish',
 'DP05_0050PE': 'Percent_RACE_One race_Native Hawaiian and Other Pacific Islander_Samoan',
 'DP04_0047E': 'Estimate_HOUSING TENURE_Occupied housing units_Renter-occupied',
 'DP02PR_0083PE': 'Percent_RESIDENCE 1 YEAR AGO_Population 1 year and over_Different house in Puerto Rico or the U.S._In Puerto Rico_Different municipio',
 'DP02PR_0106PE': 'Percent_WORLD REGION OF BIRTH OF FOREIGN BORN_Foreign-born population, excluding population born at sea_Africa',
 'DP04_0046E': 'Estimate_HOUSING TENURE_Occupied housing units_Owner-occupied',
 'DP04_0016PE': 'Percent_YEAR STRUCTURE BUILT_Total housing units',
 'DP02_0125E': 'Estimate_ANCESTRY_Total population_Czech',
 'DP02_0054PE': 'Percent_SCHOOL ENROLLMENT_Population 3 years and over enrolled in school_Kindergarten',
 'DP04_0049

Creating a list of all the column names in the ACS data that we will use cycle through to compare to the headers dictionary.

In [9]:
cols=list(acs)
cols

['Unnamed: 0',
 'GEO_ID',
 'DP03_0001E',
 'DP03_0001M',
 'DP03_0001PE',
 'DP03_0001PM',
 'DP03_0002E',
 'DP03_0002M',
 'DP03_0002PE',
 'DP03_0002PM',
 'DP03_0003E',
 'DP03_0003M',
 'DP03_0003PE',
 'DP03_0003PM',
 'DP03_0004E',
 'DP03_0004M',
 'DP03_0004PE',
 'DP03_0004PM',
 'DP03_0005E',
 'DP03_0005M',
 'DP03_0005PE',
 'DP03_0005PM',
 'DP03_0006E',
 'DP03_0006M',
 'DP03_0006PE',
 'DP03_0006PM',
 'DP03_0007E',
 'DP03_0007M',
 'DP03_0007PE',
 'DP03_0007PM',
 'DP03_0008E',
 'DP03_0008M',
 'DP03_0008PE',
 'DP03_0008PM',
 'DP03_0009E',
 'DP03_0009M',
 'DP03_0009PE',
 'DP03_0009PM',
 'DP03_0010E',
 'DP03_0010M',
 'DP03_0010PE',
 'DP03_0010PM',
 'DP03_0011E',
 'DP03_0011M',
 'DP03_0011PE',
 'DP03_0011PM',
 'DP03_0012E',
 'DP03_0012M',
 'DP03_0012PE',
 'DP03_0012PM',
 'DP03_0013E',
 'DP03_0013M',
 'DP03_0013PE',
 'DP03_0013PM',
 'DP03_0014E',
 'DP03_0014M',
 'DP03_0014PE',
 'DP03_0014PM',
 'DP03_0015E',
 'DP03_0015M',
 'DP03_0015PE',
 'DP03_0015PM',
 'DP03_0016E',
 'DP03_0016M',
 'DP03_0016PE'

In [10]:
len(cols)

1103

Removing attributes from the headers. See https://api.census.gov/data/2018/acs/acs1/profile/variables.html that the actual variables end with "E" or "PE". The attributes end with a variety of letter combinations. At the lowest level, the attributes can be removed by searching for "EA" and "M" to remove them.

In [11]:
cols=[i for i in cols if "EA" not in i]
len(cols)

829

In [12]:
cols=[i for i in cols if "M" not in i]
len(cols)

280

Creating another dictionary. Looping through the list of column names from the ACS dataset and if they are in the headers dictionary, placing them in the new dictionary that will contain only headers in the ACS dataset.

In [13]:
dp03={}
for k, v in headers_d.items():
    if k in cols:
        dp03[k]=v
dp03

{'Unnamed: 0': 'label',
 'DP03_0048PE': 'Percent_CLASS OF WORKER_Civilian employed population 16 years and over_Government workers',
 'DP03_0022PE': 'Percent_COMMUTING TO WORK_Workers 16 years and over_Walked',
 'DP03_0111PE': 'Percent_HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Unemployed_With health insurance coverage_With private health insurance',
 'DP03_0109E': 'Estimate_HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Unemployed',
 'DP03_0035PE': 'Percent_INDUSTRY_Civilian employed population 16 years and over_Manufacturing',
 'DP03_0066PE': 'Percent_INCOME AND BENEFITS (IN 2015 INFLATION-ADJUSTED DOLLARS)_With Social Security',
 'DP03_0107E': 'Estimate_HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population 18 to 64 years_In labor force_Employed_With health insurance coverage_With public coverage',
 'DP03_0108E': 'Estimate_HEALTH INSURANCE COVERAGE_Civilian noninstitut

In [14]:
len(dp03)

276

Removing the attributes.

In [15]:
acs=acs[cols]
acs.columns

Index(['Unnamed: 0', 'GEO_ID', 'DP03_0001E', 'DP03_0001PE', 'DP03_0002E',
       'DP03_0002PE', 'DP03_0003E', 'DP03_0003PE', 'DP03_0004E', 'DP03_0004PE',
       ...
       'DP03_0135E', 'DP03_0135PE', 'DP03_0136E', 'DP03_0136PE', 'DP03_0137E',
       'DP03_0137PE', 'state', 'county', 'tract', 'year'],
      dtype='object', length=280)

Renaming the columns.

In [16]:
acs.rename(columns=dp03,inplace=True)
acs.head()

/Users/sarahmartin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,label,Geography,Estimate_EMPLOYMENT STATUS_Population 16 years and over,Percent_EMPLOYMENT STATUS_Population 16 years and over,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Estimate_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force,Percent_EMPLOYMENT STATUS_Civilian labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Percent_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Estimate_EMPLOYMENT STATUS_Females 16 years and over,Percent_EMPLOYMENT STATUS_Females 16 years and over,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years,Percent_EMPLOYMENT STATUS_Own children of the householder under 6 years,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Percent_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Percent_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Percent_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Estimate_COMMUTING TO WORK_Workers 16 years and over,Percent_COMMUTING TO WORK_Workers 16 years and over,"Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Percent_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled","Percent_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled",Estimate_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Percent_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Estimate_COMMUTING TO WORK_Workers 16 years and over_Walked,Percent_COMMUTING TO WORK_Workers 16 years and over_Walked,Estimate_COMMUTING TO WORK_Workers 16 years and over_Other means,Percent_COMMUTING TO WORK_Workers 16 years and over_Other means,Estimate_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Percent_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Estimate_COMMUTING TO WORK_Mean travel time to work (minutes),Percent_COMMUTING TO WORK_Mean travel time to work (minutes),Estimate_OCCUPATION_Civilian employed population 16 years and over,Percent_OCCUPATION_Civilian employed popula

Merging State, County, and Tract columns into a single variable that can be compared to the COI GEOID. See support for merge here: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html

In [17]:
#cast as strings and then join
for i in ['state','county','tract']:
    acs[i] = acs[i].astype(str)
acs['FIPS'] = acs[['state', 'county', 'tract']].agg(''.join, axis=1)
acs['FIPS']=acs['FIPS'].astype(int)

<ipython-input-17-957d7549bf40>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acs[i] = acs[i].astype(str)


In [18]:
acs.head()

,label,Geography,Estimate_EMPLOYMENT STATUS_Population 16 years and over,Percent_EMPLOYMENT STATUS_Population 16 years and over,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Estimate_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force,Percent_EMPLOYMENT STATUS_Civilian labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Percent_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Estimate_EMPLOYMENT STATUS_Females 16 years and over,Percent_EMPLOYMENT STATUS_Females 16 years and over,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years,Percent_EMPLOYMENT STATUS_Own children of the householder under 6 years,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Percent_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Percent_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Percent_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Estimate_COMMUTING TO WORK_Workers 16 years and over,Percent_COMMUTING TO WORK_Workers 16 years and over,"Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Percent_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled","Percent_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled",Estimate_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Percent_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Estimate_COMMUTING TO WORK_Workers 16 years and over_Walked,Percent_COMMUTING TO WORK_Workers 16 years and over_Walked,Estimate_COMMUTING TO WORK_Workers 16 years and over_Other means,Percent_COMMUTING TO WORK_Workers 16 years and over_Other means,Estimate_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Percent_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Estimate_COMMUTING TO WORK_Mean travel time to work (minutes),Percent_COMMUTING TO WORK_Mean travel time to work (minutes),Estimate_OCCUPATION_Civilian employed population 16 years and over,Percent_OCCUPATION_Civilian employed popula

Uploading to S3.

In [19]:
with open('/Volumes/Key/aws_keys.json') as f:
  creds = json.load(f)

In [20]:
def s3upload(name,df):
    x = df.to_csv(None).encode()
    fs = s3fs.S3FileSystem(key=creds['AWSAccessKeyId'], secret=creds['AWSSecretKey'])
    with fs.open('s3://bleeding-hearts/workingdata/' + name + '.csv', 'wb') as f:
        f.write(x)
    print('saved')

In [21]:
s3upload('acs_20102015',acs)

saved


# COI 
Open the COI Index file

In [22]:
coi_index = pd.read_csv('s3://bleeding-hearts/rawdata/coi_index.csv')
coi_index.head()

,Unnamed: 0,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,z_ED_nat,z_HE_nat,z_SE_nat,z_COI_nat,c5_ED_nat,c5_HE_nat,c5_SE_nat,c5_COI_nat,r_ED_nat,r_HE_nat,r_SE_nat,r_COI_nat,c5_ED_stt,c5_HE_stt,c5_SE_stt,c5_COI_stt,r_ED_stt,r_HE_stt,r_SE_stt,r_COI_stt,c5_ED_met,c5_HE_met,c5_SE_met,c5_COI_met,r_ED_met,r_HE_met,r_SE_met,r_COI_met
0,0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,0.001198,-0.016530,0.046101,0.003759,Moderate,Very Low,Moderate,Moderate,55,20,54,49,High,Low,High,High,67,33,67,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,0.008932,0.035614,0.146657,0.021866,Moderate,High,High,High,59,66,75,71,High,Very High,Very High,Very High,71,92,83,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,0.013306,-0.022400,-0.133538,-0.016473,High,Very Low,Low,Low,62,17,23,27,High,Low,Low,Low,73,28,31,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,-0.003518,-0.012027,-0.169899,-0.021479,Moderate,Low,Very Low,Low,53,23,18,23,High,Low,Low,Low,64,39,26,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,-0.014533,-0.026710,0.102040,0.007462,Moderate,Very Low,High,Moderate,46,15,66,54,Moderate,Low,High,High,59,23,77,69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
len(coi_index)

144408

## COI Headers

Read in a PDF version of this webpage, which includes the data dictionary: https://data.diversitydatakids.org/dataset/coi20-child-opportunity-index-2-0-database/resource/080cfe52-90aa-4925-beaa-90efb04ab7fb. Use the data dictionary to replace the headers with descriptions.

In [24]:
tbl_lst = tabula.read_pdf('/Users/sarahmartin/Documents/GT/COI_index_headers.pdf',
                         pages="all",
                         multiple_tables=True,
                         stream=True)
print(len(tbl_lst))

6


In [25]:
#concat tables from the above list and drop blank/unnessary rows
index_head = pd.concat([tbl_lst[2],tbl_lst[3],tbl_lst[4],tbl_lst[5]])
index_head.columns = index_head.iloc[0]
index_head.drop(index_head.index[0],inplace=True)
index_head = index_head[index_head['Label'].notna()]
index_head.reset_index(inplace=True,drop=True)
index_head.head(25)

,Column,Type,Label,Description
0,geoid,text,2010 census tract FIPS codes,NaN
1,year,numeric,Year of observation (2010 or,NaN
2,NaN,NaN,2015),NaN
3,in100,numeric,In one of the 100 largest metro,This variable is equal to 1 if the
4,NaN,NaN,areas,census tract is located in one of
5,msaid15,text,Metro/Micro Area (2015) FIPS,NaN
6,NaN,NaN,Code,NaN
7,msaname15,text,Metro/Micro Area (2015) Name,NaN
8,statefips,text,State FIPS Code,NaN
9,stateusps,text,State USPS Code,NaN


In [26]:
#transpose the dataset and drop unnessary columns
transp = index_head.drop(columns=['Type','Description'])
lst = list(transp.index[transp['Column'].isnull()])
transp.drop(columns=['Column'],inplace=True)
#transpose the dataframe back. Merge sequential rows that read in separately where line breaks occurred in the PDF.
transp = transp.T
for i in reversed(lst):
    transp[i-1] = transp[i-1] + ' ' + transp[i]   
#drop unnessary columns and transpose again.
transp.drop(columns=lst,inplace=True)
transp = transp.T
transp.head()

,Label
0,2010 census tract FIPS codes
1,Year of observation (2010 or 2015)
3,In one of the 100 largest metro areas
5,Metro/Micro Area (2015) FIPS Code
7,Metro/Micro Area (2015) Name


In [27]:
#drop columns and merge index_head and transp
index_head = index_head[index_head['Column'].notna()]
index_head.drop(columns='Label',inplace=True)
index_head = index_head.merge(transp, how='outer', left_index=True, right_index=True)
index_head

,Column,Type,Description,Label
0,geoid,text,NaN,2010 census tract FIPS codes
1,year,numeric,NaN,Year of observation (2010 or 2015)
3,in100,numeric,This variable is equal to 1 if the,In one of the 100 largest metro areas
5,msaid15,text,NaN,Metro/Micro Area (2015) FIPS Code
7,msaname15,text,NaN,Metro/Micro Area (2015) Name
8,statefips,text,NaN,State FIPS Code
9,stateusps,text,NaN,State USPS Code
10,pop,numeric,NaN,"Number of children aged 0-17, ACS (2008-12 for..."
13,z_ED_nat,numeric,Weighted average of education,"Z-scores, education domain, nationally-normed"
15,z_HE_nat,numeric,Weighted average of health and,"Z-scores, health and environment domain, natio..."


In [28]:
#drop columns, reset index, and transpose -- preparing data to turn it into a dictionary.
index_head.drop(columns=['Type','Description'],inplace=True)
index_head.set_index('Column',inplace=True)
index_head=index_head.T
index_head

Column,geoid,year,in100,msaid15,msaname15,statefips,stateusps,pop,z_ED_nat,z_HE_nat,z_SE_nat,z_COI_nat,c5_ED_nat,c5_HE_nat,c5_SE_nat,c5_COI_nat,r_ED_nat,r_HE_nat,r_SE_nat,r_COI_nat,c5_ED_stt,c5_HE_stt,c5_SE_stt,c5_COI_stt,r_ED_stt,r_HE_stt,r_SE_stt,r_COI_stt,c5_ED_met,c5_HE_met,c5_SE_met,c5_COI_met,r_ED_met,r_HE_met,r_SE_met,r_COI_met
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Label,2010 census tract FIPS codes,Year of observation (2010 or 2015),In one of the 100 largest metro areas,Metro/Micro Area (2015) FIPS Code,Metro/Micro Area (2015) Name,State FIPS Code,State USPS Code,"Number of children aged 0-17, ACS (2008-12 for...","Z-scores, education domain, nationally-normed","Z-scores, health and environment domain, natio...","Z-scores, social and economic domain, national...","Z-scores, overall COI, nationally-normed","Child Opportunity Levels, education domain, na...","Child Opportunity Levels, health and environme...","Child Opportunity Levels, social and economic ...","Child Opportunity Levels, overall COI, nationa...","Child Opportunity Scores, education domain, na...","Child Opportunity Scores, health and environme...","Child Opportunity Scores, social and economic ...","Child Opportunity Scores, overall COI, nationa...","Child Opportunity Levels, education domain, st...","Child Opportunity Levels, health and environme...","Child Opportunity Levels, social and economic ...","Child Opportunity Levels, overall COI, state-n...","Child Opportunity Scores, education domain, st...","Child Opportunity Scores, health and environme...","Child Opportunity Scores, social and economic ...","Child Opportunity Scores, overall COI, state-n...","Child Opportunity Levels, education domain, me...","Child Opportunity Levels, health and environme...","Child Opportunity Levels, social and economic ...","Child Opportunity Levels, overall COI, metro-n...","Child Opportunity Scores, education domain, me...","Child Opportunity Scores, health and environme...","Child Opportunity Scores, social and economic ...","Child Opportunity Scores, overall COI, metro-n..."


In [29]:
#transform data into a dictionary
index_head_d = index_head.to_dict('records')
index_head_d = index_head_d[0]
index_head_d

{'geoid': '2010 census tract FIPS codes',
 'year': 'Year of observation (2010 or 2015)',
 'in100': 'In one of the 100 largest metro areas',
 'msaid15': 'Metro/Micro Area (2015) FIPS Code',
 'msaname15': 'Metro/Micro Area (2015) Name',
 'statefips': 'State FIPS Code',
 'stateusps': 'State USPS Code',
 'pop': 'Number of children aged 0-17, ACS (2008-12 for 2010, 2013-17 for 2015)',
 'z_ED_nat': 'Z-scores, education domain, nationally-normed',
 'z_HE_nat': 'Z-scores, health and environment domain, nationally-normed',
 'z_SE_nat': 'Z-scores, social and economic domain, nationally-normed',
 'z_COI_nat': 'Z-scores, overall COI, nationally-normed',
 'c5_ED_nat': 'Child Opportunity Levels, education domain, nationally- normed',
 'c5_HE_nat': 'Child Opportunity Levels, health and environment domain, nationally-normed',
 'c5_SE_nat': 'Child Opportunity Levels, social and economic domain, nationally-normed',
 'c5_COI_nat': 'Child Opportunity Levels, overall COI, nationally-normed',
 'r_ED_nat': '

In [30]:
#use the dictionary to rename the columns
coi_index.rename(columns=index_head_d,inplace=True)
coi_index

,Unnamed: 0,2010 census tract FIPS codes,Year of observation (2010 or 2015),In one of the 100 largest metro areas,Metro/Micro Area (2015) FIPS Code,Metro/Micro Area (2015) Name,countyfips,State FIPS Code,State USPS Code,"Number of children aged 0-17, ACS (2008-12 for 2010, 2013-17 for 2015)","Z-scores, education domain, nationally-normed","Z-scores, health and environment domain, nationally-normed","Z-scores, social and economic domain, nationally-normed","Z-scores, overall COI, nationally-normed","Child Opportunity Levels, education domain, nationally- normed","Child Opportunity Levels, health and environment domain, nationally-normed","Child Opportunity Levels, social and economic domain, nationally-normed","Child Opportunity Levels, overall COI, nationally-normed","Child Opportunity Scores, education domain, nationally- normed","Child Opportunity Scores, health and environment domain, nationally-normed","Child Opportunity Scores, social and economic domain, nationally-normed","Child Opportunity Scores, overall COI, nationally-normed","Child Opportunity Levels, education domain, state- normed","Child Opportunity Levels, health and environment domain, state-normed","Child Opportunity Levels, social and economic domain, state- normed","Child Opportunity Levels, overall COI, state-normed","Child Opportunity Scores, education domain, state- normed","Child Opportunity Scores, health and environment domain, state-normed","Child Opportunity Scores, social and economic domain, state-normed","Child Opportunity Scores, overall COI, state-normed","Child Opportunity Levels, education domain, metro- normed","Child Opportunity Levels, health and environment domain, metro-normed","Child Opportunity Levels, social and economic domain, metro- normed","Child Opportunity Levels, overall COI, metro-normed","Child Opportunity Scores, education domain, metro- normed","Child Opportunity Scores, health and environment domain, metro-normed","Child Opportunity Scores, social and economic domain, metro-normed","Child Opportunity Scores, overall COI, metro-normed"
0,0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,0.001198,-0.016530,0.046101,0.003759,Moderate,Very Low,Moderate,Moderate,55,20,54,49,High,Low,High,High,67,33,67,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,0.008932,0.035614,0.146657,0.021866,Moderate,High,High,High,59,66,75,71,High,Very High,Very High,Very High,71,92,83,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,0.013306,-0.022400,-0.133538,-0.016473,High,Very Low,Low,Low,62,17,23,27,High,Low,Low,Low,73,28,31,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,-0.003518,-0.012027,-0.169899,-0.021479,Moderate,Low,Very Low,Low,53,23,18,23,High,Low,Low,Low,64,39,26,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,-0.014533,-0.026710,0.102040,0.007462,Moderate,Very Low,High,Moderate,46,15,66,54,Moderate,Low,High,High,59,23,77,69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144403,144403,56043000302,2015,NaN,NaN,NaN,56043,56,WY,706,0.023999,0.026997,0.063540,0.012929,High,Moderate,Moderate,Moderate,67,56,58,60,High,Low,Low,Moderate,77,25,33,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144404,144404,56045951100,2010,NaN,NaN,NaN,56045,56,WY,638,-0.028395,0.050417,-0.073775,-0.006410,Low,High,Low,Low,38,80,31,37,Low,High,Very Low,Very Low,27,62,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144405,144405,56045951100,2015,NaN,NaN,NaN,56045,56,WY,577,-0.009124,0.038326,-0.053315,-0.003169,Moderate,High,Low,Moderate,49,68,35,41,Moderate,Low,Very Low,Very Low,45,39,8,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144406,144406,56045951300,2010,NaN,NaN,NaN,56045,56,WY,954,-0.004

In [31]:
s3upload('coi_index',coi_index)

saved


## Merge Datasets

In [32]:
#merge on both the FIPS code and the year
merge = pd.merge(acs, 
                  coi_index,
                 left_on=['FIPS','year'],
                  right_on=['2010 census tract FIPS codes','Year of observation (2010 or 2015)'])

In [33]:
merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18897 entries, 0 to 18896
Columns: 319 entries, label to Child Opportunity Scores, overall COI, metro-normed
dtypes: float64(113), int64(188), object(18)
memory usage: 46.1+ MB


In [34]:
merge.tail()

,label,Geography,Estimate_EMPLOYMENT STATUS_Population 16 years and over,Percent_EMPLOYMENT STATUS_Population 16 years and over,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Estimate_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force,Percent_EMPLOYMENT STATUS_Civilian labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Percent_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Estimate_EMPLOYMENT STATUS_Females 16 years and over,Percent_EMPLOYMENT STATUS_Females 16 years and over,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years,Percent_EMPLOYMENT STATUS_Own children of the householder under 6 years,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Percent_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Percent_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Percent_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Estimate_COMMUTING TO WORK_Workers 16 years and over,Percent_COMMUTING TO WORK_Workers 16 years and over,"Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Percent_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled","Percent_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled",Estimate_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Percent_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Estimate_COMMUTING TO WORK_Workers 16 years and over_Walked,Percent_COMMUTING TO WORK_Workers 16 years and over_Walked,Estimate_COMMUTING TO WORK_Workers 16 years and over_Other means,Percent_COMMUTING TO WORK_Workers 16 years and over_Other means,Estimate_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Percent_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Estimate_COMMUTING TO WORK_Mean travel time to work (minutes),Percent_COMMUTING TO WORK_Mean travel time to work (minutes),Estimate_OCCUPATION_Civilian employed population 16 years and over,Percent_OCCUPATION_Civilian employed popula

In [ ]:
s3upload('merge',merge)